In [1]:
import polars as pl
import pandas as pd
import re

from tools.josiedataprep.preparation_functions import _create_set_with_bag_semantic, _create_set_with_set_semantic

In [4]:
# root_dir = '/home/nanni/unimore/tesi-magistrale/data/josie-tests/n45673_mbag'
root_dir = '/home/nanni/unimore/tesi-magistrale/data/josie-tests/n45673_mset'


tables_csv_dir =        '/home/nanni/unimore/tesi-magistrale/data/turl_sloth_wikitables/tables-subset/csv'
josie_sloth_ids_file =  root_dir + '/josie_sloth_ids.csv'
tables_file =           root_dir + '/tables.set'

In [5]:
josie_sloth_ids = pd.read_csv(josie_sloth_ids_file)

In [7]:
# josie_res = pd.read_csv('/home/nanni/unimore/tesi-magistrale/data/josie-tests/n45673_mbag/result_k_5.csv')[['query_id', 'results']]

josie_res = pd.read_csv('/home/nanni/unimore/tesi-magistrale/data/josie-tests/n45673_mset/result_k_5.csv')[['query_id', 'results']]
josie_res

,query_id,results
0,47384,s38722o1158s56253o603s18772o540s48260o56s45737o55
1,68509,s69203o45s68747o45s38674o43s11177o34s8287o28
2,24796,s69310o29s32525o24s54512o23s49194o22s25478o22
3,2637,s40234o46s78670o46s7633o45s22982o45s77978o44
4,62436,s8819o29s86983o29s45911o29s2586o29s87353o29
...,...,...
89,3362,s45755o33s77503o33s47295o33s8178o32s9564o32
90,25305,s31472o73s1893o73s61522o73s62689o73s84146o73
91,79547,s40234o27s85111o27s3416o27s61368o27s25960o27
92,83910,s56686o95s64367o95s40010o95s3069o66s62588o66


In [9]:
print('query_id', 'set_id ', 'josie_o', 'my_o')
for row in josie_res.itertuples(index=False):
    qid = row[0]
    if type(row[1]) != str:
        continue
    res = row[1]
    sids, overlaps = re.findall(r'\d+', row[1])[::2], re.findall(r'\d+', row[1])[1::2]
    s_id1 = josie_sloth_ids[josie_sloth_ids['josie_id'] == qid]['sloth_id'].values[0]
    
    for sid, o in zip(sids, overlaps):                
        s_id2 = josie_sloth_ids[josie_sloth_ids['josie_id'] == int(sid)]['sloth_id'].values[0]

        df1 = pd.read_csv(tables_csv_dir + '/' + s_id1)
        df2 = pd.read_csv(tables_csv_dir + '/' + s_id2)

        set1 = _create_set_with_set_semantic(df1)
        set2 = _create_set_with_set_semantic(df2)

        print(qid, '\t', sid, '\t', o, '\t', len(set1.intersection(set2)))
        if abs(len(set1.intersection(set2)) - int(o)) not in (0, 1):
            print('Errore ', s_id1, s_id2)

query_id set_id  josie_o my_o
47384 	 38722 	 1158 	 1158
47384 	 56253 	 603 	 603
47384 	 18772 	 540 	 540
47384 	 48260 	 56 	 56
47384 	 45737 	 55 	 55
68509 	 69203 	 45 	 45
68509 	 68747 	 45 	 45
68509 	 38674 	 43 	 43
68509 	 11177 	 34 	 34
68509 	 8287 	 28 	 28
24796 	 69310 	 29 	 29
24796 	 32525 	 24 	 24
24796 	 54512 	 23 	 23
24796 	 49194 	 22 	 22
24796 	 25478 	 22 	 22
2637 	 40234 	 46 	 47
2637 	 78670 	 46 	 47
2637 	 7633 	 45 	 46
2637 	 22982 	 45 	 46
2637 	 77978 	 44 	 45
62436 	 8819 	 29 	 29
62436 	 86983 	 29 	 29
62436 	 45911 	 29 	 29
62436 	 2586 	 29 	 29
62436 	 87353 	 29 	 29
56235 	 71467 	 65 	 66
56235 	 33659 	 47 	 48
56235 	 56726 	 46 	 47
56235 	 16267 	 46 	 47
56235 	 71923 	 46 	 47
76524 	 87191 	 83 	 84
76524 	 15808 	 83 	 84
76524 	 17022 	 82 	 83
76524 	 47295 	 82 	 83
76524 	 45755 	 82 	 83
69763 	 8038 	 1298 	 1298
69763 	 76600 	 1245 	 1245
69763 	 15779 	 1208 	 1208
69763 	 2833 	 1138 	 1138
69763 	 46562 	 1070 

In [20]:
df1 = pd.read_csv(tables_csv_dir + f'/30476492-2')
df2 = pd.read_csv(tables_csv_dir + f'/11383852-3')

In [22]:
df1.convert_dtypes()

,Rank,Nation,Gold,Silver,Bronze,Total
0,1,China,10,4,0,14
1,2,[[|]],0,2,1,3
2,3,[[|]],0,1,3,4
3,4,Australia,0,1,1,2
4,4,Canada,0,1,1,2
5,6,United States,0,1,0,1
6,7,[[|]],0,0,2,2
7,8,Italy,0,0,1,1
8,8,[[|]],0,0,1,1
9,Total,Total,10,10,10,30


In [17]:
df2

,Team,1979,1983,1987,1991,1995,1999,2003,2007,2009,2011,2013,Total
0,[[|]],NaN,13th,NaN,13th,14th,NaN,13th,NaN,14th,NaN,NaN,5.0
1,[[|]],3rd,7th,NaN,3rd,6th,4th,10th,6th,5th,4th,12th,10.0
2,Australia,9th,10th,5th,11th,2nd,5th,1st,5th,4th,6th,4th,11.0
3,[[|]],2nd,3rd,10th,7th,NaN,8th,NaN,4th,NaN,9th,10th,8.0
4,Canada,7th,14th,6th,8th,NaN,NaN,NaN,10th,7th,11th,6th,8.0
5,China,NaN,11th,9th,10th,9th,15th,14th,12th,NaN,13th,7th,9.0
6,Chinese Taipei,NaN,NaN,11th,NaN,NaN,NaN,NaN,NaN,NaN,14th,NaN,2.0
7,[[|]],NaN,NaN,NaN,NaN,4th,3rd,4th,NaN,3rd,8th,8th,6.0
8,[[|]],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14th,1.0
9,[[|]],NaN,9th,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
